### Runable
* Runnable은 런타임에 실행될수있는 모든 객체를 의미한다

### Runnable 종류
* invoke :입력에 대해 체인을 호출하는 메서드
* stream :응답의 청크를 스트리밍한다
* batch :입력목록에 대한 일괄처리를 수행한다.

이외에 비동기 메소드도 존재한다.
* astream : 비동기적으로 응답의 청크를 비동기 스트리밍한다.
* ainvoke : 비동기적으로 입력에 대해 체인을 호출한다.
* abatch : 비동기적으로 입력 목록에 대해 일괄 처리를 수행한다.
* astream_log : 최종 응답 및 발생하는 중간단계를 스트리밍한다.

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.1
)

In [24]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{lecture}에 대해 3문장으로 설명해줘")

chain = prompt | llm | StrOutputParser()

In [25]:
print(chain.invoke({"lecture" : "리엑트"}));

리액트(React)는 사용자 인터페이스를 구축하기 위한 자바스크립트 라이브러리로, 컴포넌트 기반 아키텍처를 통해 재사용 가능한 UI 요소를 쉽게 만들 수 있게 해줍니다. 리액트는 가상 DOM(Virtual DOM)을 사용하여 실제 DOM 변경을 최소화하고, 애플리케이션의 성능을 최적화합니다. 또한, 리액트는 단방향 데이터 흐름을 채택하여 데이터의 예측 가능성과 디버깅을 용이하게 합니다.


### strean [실시간 출력]
* 데이터 스트림을 생성하고 , 스트림을 반복하여 각 데이터의 즉시 출력한다
* end=" 인자를 사용해 줄바꿈을 하지 않을수있다
* flush=True 인자를 사용해 즉시 출력가능

In [26]:
for token in chain.stream({"lecture" : "Python"}):

    print(token, end="", flush=True)

Python은 높은 수준의 프로그래밍 언어로, 읽기 쉽고 간결한 문법을 제공하여 초보자와 전문가 모두에게 인기가 많습니다. 다양한 라이브러리와 프레임워크를 통해 웹 개발, 데이터 분석, 인공지능 등 여러 분야에서 활용됩니다. 또한, 플랫폼에 독립적이며 커뮤니티의 활발한 지원을 받아 지속적으로 발전하고 있습니다.

### batch [단위 실행]
* 여러개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각각의 입력에 대한 체인을 실행한다
* max_concurrency :최대 병렬처리수를 지정할수있다.

In [27]:
result = chain.batch(
    [
        {"lecture": "파이썬"},
        {"lecture": "자바"},
        {"lecture": "자바스크립트"},
        {"lecture": "SQL"}
    ],
    config={"max_concurrency": 3}
)

print(result)

['파이썬은 높은 수준의 프로그래밍 언어로, 읽기 쉽고 간결한 문법을 제공하여 초보자와 전문가 모두에게 인기가 많습니다. 다양한 라이브러리와 프레임워크를 통해 웹 개발, 데이터 분석, 인공지능 등 여러 분야에서 활용됩니다. 또한, 플랫폼에 독립적이며 커뮤니티 지원이 활발하여 지속적으로 발전하고 있습니다.', '자바(Java)는 객체 지향 프로그래밍 언어로, 1995년 썬 마이크로시스템즈(Sun Microsystems)에서 처음 개발되었습니다. 플랫폼 독립성을 제공하여 다양한 운영 체제에서 실행할 수 있으며, "한 번 작성하면 어디서나 실행된다"는 철학을 가지고 있습니다. 자바는 웹 애플리케이션, 모바일 앱, 엔터프라이즈 소프트웨어 등 다양한 분야에서 널리 사용되고 있습니다.', '자바스크립트는 웹 개발에서 널리 사용되는 프로그래밍 언어로, 웹 페이지에 동적인 기능을 추가하는 데 사용됩니다. 클라이언트 측에서 실행되며, HTML 및 CSS와 함께 웹의 기본적인 구성 요소 중 하나로 작동합니다. 또한, Node.js와 같은 런타임 환경을 통해 서버 측 프로그래밍에도 활용될 수 있습니다.', 'SQL(Structured Query Language)은 관계형 데이터베이스 관리 시스템(RDBMS)에서 데이터를 관리하고 조작하기 위한 표준 프로그래밍 언어입니다. SQL을 사용하면 데이터베이스에서 데이터를 검색, 삽입, 업데이트 및 삭제할 수 있으며, 데이터베이스 구조를 정의하고 제어할 수도 있습니다. SQL은 다양한 데이터베이스 시스템에서 사용되며, 데이터 쿼리 및 분석을 위한 강력한 도구로 널리 활용됩니다.']


---
parallel(병렬성)
* LCEL 를 사용하여 체인을 구성할때, 여러 체인을 동시해 실행할수있다.



In [28]:
chain1 =(
    PromptTemplate.from_template("{country}의 수도는 어디야?")
    | llm
    | StrOutputParser()
)

chain2 =(
    PromptTemplate.from_template("{country}의 면적은 얼마야")
    | llm
    | StrOutputParser()
)

In [29]:
from langchain_core.runnables import RunnableParallel

# 병렬 실행 체인
combined = RunnableParallel(capital=chain1, area=chain2)

In [30]:
print(chain1.invoke({"country" : "대한민국"}))

대한민국의 수도는 서울입니다.


In [31]:
print(chain2.invoke({"country" : "미국"}))

미국의 총 면적은 약 9,833,520 제곱킬로미터입니다. 이 면적에는 50개 주와 워싱턴 D.C.가 포함됩니다. 미국은 세계에서 세 번째로 큰 나라로, 러시아와 캐나다에 이어 면적이 넓습니다.


In [32]:
result = combined.invoke({"country" :"대한민국"})

print(result)

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 면적은 약 100,210 평방 킬로미터입니다.'}


In [33]:
print(result["capital"])
print(result["area"])

대한민국의 수도는 서울입니다.
대한민국의 면적은 약 100,210 평방 킬로미터입니다.


### RunnablePassThrough
* 입력을 그대로 전달하는 역할
* 단독으로 호출될경우 단순히 입력을 받아 그대로 전달

In [9]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.0
)

prompt = PromptTemplate.from_template("{num}의 약수를 알려줘")

chain = prompt | llm | StrOutputParser()

In [10]:
# invoke()를 통해 실행할때는 입력이 딕셔너리여야하지만
# 1개의 변수만 템플릿에 작성이 되있다면, 값만 전달해도 된다
chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

In [11]:
from langchain_core.runnables import RunnablePassthrough

passthrough_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()

passthrough_chain.invoke({"num":10})

'10의 약수는 1, 2, 5, 10입니다.'

**RunnablePassthrough.assign()**
* 입력값으로 들어온 값의 key/value쌍을 새롭게 할당된 key/value쌍을 합쳐준다

In [12]:
RunnablePassthrough.assign(new_date=lambda x: x["num"] * 3).invoke({"num":1})

{'num': 1, 'new_date': 3}

### RunnableLambda
* 입력값에 대한 추가 처리를 수행할수있따
* 람다함수를 사용해 입력값에 대한 추가 처리를 진행

* 날씨 시간

In [20]:
from datetime import datetime

def get_today(num):
    print("num : ", num)
    return datetime.now().strftime("%b-%d")

# get_today()

In [21]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

#prompt
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 알려줘"
)

#llm
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

#chain생성
chain =(
    {"today" : RunnableLambda(get_today),"n": itemgetter("n")}
    | prompt
    | llm
    | StrOutputParser()
)



In [22]:
print(chain.invoke({"n": 3}))

num :  {'n': 3}
10월 28일이 생일인 유명인 중 몇 명을 소개하자면 다음과 같습니다.

1. 빌 게이츠 (Bill Gates) - 마이크로소프트의 공동 창립자이자 세계적인 기업가입니다.
2. 줄리아 로버츠 (Julia Roberts) - 아카데미상을 수상한 미국의 유명한 배우입니다.
3. 호아킨 피닉스 (Joaquin Phoenix) - 여러 영화에서 뛰어난 연기를 선보인 미국의 배우입니다.

이 외에도 많은 유명인들이 있지만, 이 세 명이 대표적입니다.
